In [1]:
import pandas as pd
import pandas_profiling as pp
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import scipy.sparse

In [2]:
data_train = pd.read_csv(r'E:\PycharmProjects\Machine_L\Data_Science_Club\Hackathon\train (1).csv')
data_test = pd.read_csv(r'E:\PycharmProjects\Machine_L\Data_Science_Club\Hackathon\test (1).csv')

c:\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_train.head()

,comment_id,company_id,user_id,product_id,comment,rating,date_created,user_date_created,user_ip,user_agent,is_fake
0,1,2169053,91933,6.0,компания не выполнила заказ без обьяснения пр...,1,2014-10-23 20:26:44,2010-02-03 21:03:22,178.92.70.220,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,False
1,2,709306,91933,2.0,"Впервые встретился такой професиональный, отве...",5,2016-12-10 08:38:27,2010-02-03 21:03:22,94.179.59.162,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,False
2,3,35287,91933,NaN,"Несерйозная, некоректная менеджер інтернет маг...",1,2017-02-11 15:55:14,2010-02-03 21:03:22,37.73.137.188,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False
3,4,1778322,91933,1.0,"Не рекомендую. Не отвечают, не выполняют заказы.",1,2017-06-26 18:00:45,2010-02-03 21:03:22,37.73.149.180,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False
4,5,2821533,91933,NaN,"На письма не отвечают, заказы не выполняют. Не...",1,2017-07-06 16:57:40,2010-02-03 21:03:22,37.73.221.126,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False


In [4]:
data_test.head()

,comment,comment_id,company_id,date_created,month_created,product_id,rating,user_agent,user_date_created,user_id,user_ip
0,"спасибо все отлично, качество как на картинке,...",1655822,2214161,2018-03-19 21:36:07,2018-03,1301549.0,5,NaN,2017-11-01 00:00:46,12969242,NaN
1,Заказывала шоколад Риттер Спорт 10 шт. К сожал...,1655823,2333068,2017-12-10 13:46:49,2017-12,699700.0,5,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2017-11-01 00:02:19,12969248,91.90.13.210
2,"Быстро отзвонились, охотно отвечали на все воп...",1655824,752692,2017-11-01 00:07:38,2017-11,NaN,5,Mozilla/5.0 (Windows NT 6.1; rv:38.0) Gecko/20...,2017-11-01 00:02:44,12969251,93.115.95.204
3,Очень оперативный и внимательный к покупателю ...,1655825,334751,2017-11-20 23:40:01,2017-11,1279247.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
4,Все прошло очень оперативно! Заказ привезли в ...,1655826,2263650,2017-11-20 23:39:51,2017-11,1225545.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28


In [5]:
data_train.corr()

,comment_id,company_id,user_id,product_id,rating,is_fake
comment_id,1.000000,0.007737,0.945726,0.802164,0.025298,0.096783
company_id,0.007737,1.000000,0.006817,0.007686,0.009180,0.020309
user_id,0.945726,0.006817,1.000000,0.753244,0.035457,0.120007
product_id,0.802164,0.007686,0.753244,1.000000,0.000280,0.103157
rating,0.025298,0.009180,0.035457,0.000280,1.000000,0.026522
is_fake,0.096783,0.020309,0.120007,0.103157,0.026522,1.000000


In [6]:
data_train.is_fake.value_counts()

False    1276801
True        4698
Name: is_fake, dtype: int64

In [7]:
data_train.loc[:, 'is_fake'] = data_train.loc[:, 'is_fake'].replace(to_replace = [False, True], value = [0, 1])

In [8]:
all_text = pd.concat([data_train['comment'], data_test['comment']])

In [9]:
'''
vectorizer_train = TfidfVectorizer(min_df = 5, max_features = 40000)
vectorizer_train.fit(data_train['comment'][: int(len(data_train) / 2)])
'''

"\nvectorizer_train = TfidfVectorizer(min_df = 5, max_features = 40000)\nvectorizer_train.fit(data_train['comment'][: int(len(data_train) / 2)])\n"

In [9]:
data_train.head()

,comment_id,company_id,user_id,product_id,comment,rating,date_created,user_date_created,user_ip,user_agent,is_fake
0,1,2169053,91933,6.0,компания не выполнила заказ без обьяснения пр...,1,2014-10-23 20:26:44,2010-02-03 21:03:22,178.92.70.220,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,0
1,2,709306,91933,2.0,"Впервые встретился такой професиональный, отве...",5,2016-12-10 08:38:27,2010-02-03 21:03:22,94.179.59.162,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,0
2,3,35287,91933,NaN,"Несерйозная, некоректная менеджер інтернет маг...",1,2017-02-11 15:55:14,2010-02-03 21:03:22,37.73.137.188,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,0
3,4,1778322,91933,1.0,"Не рекомендую. Не отвечают, не выполняют заказы.",1,2017-06-26 18:00:45,2010-02-03 21:03:22,37.73.149.180,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,0
4,5,2821533,91933,NaN,"На письма не отвечают, заказы не выполняют. Не...",1,2017-07-06 16:57:40,2010-02-03 21:03:22,37.73.221.126,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,0


In [10]:
fakes = data_train[data_train['is_fake'] == True]
pp.ProfileReport(fakes)

Number of variables,12
Number of observations,4698
Total Missing (%),1.8%
Total size in memory,348.7 KiB
Average record size in memory,76.0 B
Numeric,4
Categorical,4
Boolean,0
Date,0
Text (Unique),1
Rejected,3


In [11]:
fakes.comment.head()

3788      Огромное спасибо за посылку! Крылья бабочек-су...
151859    Спасибо Вашему магазину за качественный товар,...
151860    Спасибо за быструю доставку,за качественный то...
315022    заказываю не впервые, как впрочем и всегда рад...
315023    очень довольна обслуживание, все четко и в ого...
Name: comment, dtype: object

In [12]:
pd.Series(fakes.product_id.isnull()).value_counts()

False    3686
True     1012
Name: product_id, dtype: int64

In [13]:
vectorizer_fakes = TfidfVectorizer(min_df = 5, max_features = 20000, ngram_range = (1,3))
vectorizer_fakes.fit(fakes.comment)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=5,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [14]:
sparse_fakes = vectorizer_fakes.transform(fakes.comment)

In [15]:
reals = data_train[data_train.is_fake == False]
#sparse_reals = TfidfVectorizer(min_df = 5, max_features = 20000, ngram_range = (1,3)).fit_transform(reals.comment)

In [17]:
#sparse_matrix_train = scipy.sparse.hstack([sparse_fakes, sparse_reals])

In [110]:
both_in_train = [el for el in fakes.product_id if el in reals.product_id] #product_id in fakes and reals

In [116]:
both_in_test_and_train = [el for el in data_test.product_id if el in data_train.product_id]

In [119]:
fakes_in_test_train = [el for el in fakes.product_id if el in both_in_test_and_train and el not in both_in_train]

In [120]:
fakes_in_test_train

[1247139.0]

In [122]:
#reals_in_test_train = [el for el in reals.product_id if el in both_in_test_and_train and el not in both_in_train]

In [ ]:
company_both = [el for el in fakes.com]